In [1]:
import pandas as pd
import os
import numpy as np


In [2]:
# Ordnerpfad mit allen Daten
ordnerpfad = "C:/Users/anasl/Desktop/Daten 1/Daten/Yearly Data"

# Alle Dateien im Ordner auflisten
alle_dateien = os.listdir(ordnerpfad)


In [3]:
# Liste zum Speichern aller bereinigten DataFrames
alle_daten = []


In [4]:
for dateiname in alle_dateien:
    
    dateipfad = os.path.join(ordnerpfad, dateiname)
    
    if dateiname.endswith(".csv"):
        daten = pd.read_csv(dateipfad, sep=";", encoding="latin1")
        
        # Spalten umbenennen (nur die relevanten)
        daten = daten.rename(columns={
            "MinTemp": "MinTemp",
            "AvgTemp": "AvgTemp",
            "MaxTemp": "MaxTemp",
            "Regen": "Regen",
            "Sonne": "Sonne",
            "Standort": "Standort",
            "Datum": "Datum",
            "Boden": "Boden"
        })

    elif dateiname.endswith(".tsv"):
        daten = pd.read_csv(dateipfad, sep="\t", encoding="latin1")
        
        # Neue Spalte Boden (nicht vorhanden in .tsv)
        daten["Boden"] = np.nan
        
        # Spalten umbenennen
        daten = daten.rename(columns={
            "Standort": "Standort",
            "Datum": "Datum",
            "Min_2m": "MinTemp",
            "Mittel_2m": "AvgTemp",
            "Max_2m": "MaxTemp",
            "Niederschlagshoehe": "Regen",
            "Sonnenscheindauer": "Sonne"
        })

    else:
        continue  # Falls es keine .csv oder .tsv ist, überspringen

    alle_daten.append(daten)

In [5]:
# Alle Daten in einem DataFrame kombinieren
vereinigte_daten = pd.concat(alle_daten, ignore_index=True)

In [6]:
# Datum konvertieren
vereinigte_daten["Datum"] = pd.to_datetime(vereinigte_daten["Datum"], dayfirst=True, errors="coerce")

# Nur Daten ab 01.01.2020 behalten
vereinigte_daten = vereinigte_daten[vereinigte_daten["Datum"] >= "2020-01-01"]

In [7]:
# Boden, MinTemp, AvgTemp, MaxTemp → mit Mittelwert pro Standort auffüllen

for spalte in ["Boden", "MinTemp", "AvgTemp", "MaxTemp"]:
    vereinigte_daten[spalte] = vereinigte_daten.groupby("Standort")[spalte].transform(
        lambda x: x.fillna(x.mean())
    )

# Sonne → fehlende Werte mit 0 ersetzen
vereinigte_daten["Sonne"] = vereinigte_daten["Sonne"].fillna(0)


In [8]:
# Daten nach Datum und Standort sortieren
vereinigte_daten = vereinigte_daten.sort_values(["Datum", "Standort"])

# Datei als CSV exportieren
vereinigte_daten.to_csv("yearly_combined_1.csv", sep=",", index=False, encoding="latin1")
